### 这里是这版要用的思路，轻量版评论爬取
#### 每个景点只考虑默认的3020条评论
几个要点要考虑
1. 采用多线程池加速爬取

2. cticket两个交替随机使用 

3. 加入 fake_useragent 减少反爬

3. 每一列的变量包括 comment_id, comment_user_id, poi_id, poi_name, comment_time 和 comment_type 

4. 可以先导出评论再加入景点相关的变量 


#### 1.1 加载需要的库

In [ ]:
# -*- coding: utf-8 -*-
import re
import requests
import time
import os
import math
import ssl
import pandas as pd
from requests.adapters import HTTPAdapter
import json
import random
from fake_useragent import UserAgent 
from concurrent.futures import ThreadPoolExecutor, as_completed

#### 1.2 设置参数

In [2]:
PAGE_SIZE = 20      # 每页条数
MAX_PAGE = 300      # 最多翻多少页（防止过多）
POOL_NUMBER = 6    # 线程池数量
TIME_OUT = 10       # 超时时间

In [3]:
city_name = "北京"

# 创建城市文件夹（如果不存在）
city_folder = os.path.join("comments_result", city_name)
os.makedirs(city_folder, exist_ok=True)

In [4]:
# --------- 通用配置 ---------
HEADERS = {
    'authority': 'm.ctrip.com',
    'accept': '*/*',
    'content-type': 'application/json',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'no-cache',
    'cookieorigin': 'https://you.ctrip.com',
    'origin': 'https://you.ctrip.com',
    'pragma': 'no-cache',
    'referer': 'https://you.ctrip.com/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
}

ctickets = [
    "E89D6D1A5B866B002944528EAD21DD72A941215F8FEFEA4DC289217D0F196468",
    "A7F9A3EFD9AA82FFB0032491AE748FEDE7F3C0CDE1D26A99545E319742E6D2BD"
]

class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        kwargs['ssl_version'] = ssl.PROTOCOL_TLSv1_2
        return super().init_poolmanager(*args, **kwargs)


In [5]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#### 1.3 当个景点的爬取函数（类）

##### 制作一个默认版本（提取主题和关键词）一个时间版本（分析情感分布）

In [7]:
# 单个景点评论获取函数
class SpiderCtripComments:
    def __init__(self, poi_id: int, name: str, folder: str,
                 poi_type: str = "", poi_subtype: str = "",
                 ctickets: list = None, sleep_time: float = 0.3):
        # 清理非法字符，保证文件名安全

        
        self.poi_id = poi_id
        self.name = name
        self.poi_type = poi_type
        self.poi_subtype = poi_subtype
        safe_name = re.sub(r'[\\/:*?"<>|]', "_", name)
        self.file_path = os.path.join(folder, f"{safe_name}.csv")  # ✅ 直接拼文件名
        os.makedirs(folder, exist_ok=True)
        self.ua = UserAgent()
        self.ctickets = ctickets or []
        self.sleep_time = sleep_time
        self.seen_comment_ids = set()

        # 加载历史评论 ID 做去重
        if os.path.exists(self.file_path):
            try:
                old_df = pd.read_csv(self.file_path, usecols=["commentId"])
                self.seen_comment_ids = set(old_df["commentId"].astype(str).tolist())
                print(f"📂 已加载 {len(self.seen_comment_ids)} 条历史评论用于去重")
            except Exception as e:
                print(f"⚠️ 载入已有 CSV 出错: {e}")

    def get_next_cookie(self):
        if not self.ctickets:
            return {}
        cticket = random.choice(self.ctickets)
        return {"cticket": cticket}

    def generate_params(self):
        guid = str(int(time.time() * 1000000))
        random_number = random.randint(100000, 999999)
        return {
            "_fxpcqlniredt": guid,
            "x-traceID": f"{guid}-{int(time.time() * 1000000)}-{random_number}"
        }

    def get_single_page_comments(self, page_index: int) -> list:
        url = "https://m.ctrip.com/restapi/soa2/13444/json/getCommentCollapseList"
        params = self.generate_params()
        data = {
            "arg": {
                "channelType": 2,
                "collapseType": 0,
                "commentTagId": 0,
                "pageIndex": page_index,
                "pageSize": PAGE_SIZE,
                "poiId": int(self.poi_id),
                "resourceType": 11,
                "sortType": 3, # 3默认，系统推荐有代表性 #1时间顺序
                "sourceType": 1,
                "starType": 0,
            },
            "head": {
                "auth": "",
                "cid": params["_fxpcqlniredt"],
                "ctok": "",
                "cver": "1.0",
                "lang": "01",
                "sid": "8888",
                "syscode": "09",
                "xsid": "",
            }
        }

        for attempt in range(3):  # 最多尝试 3 次
            try:
                res = requests.post(
                    url=url,
                    params=params,
                    headers={"User-Agent": self.ua.random},
                    cookies=self.get_next_cookie(),
                    json=data,
                    timeout=TIME_OUT,
                    verify=False  # 避免偶发 SSL 错误
                )
                res.raise_for_status()
                time.sleep(self.sleep_time + random.uniform(0.1, 0.4))  # 模拟用户操作停顿
                return res.json().get("result", {}).get("items", []) or []
            except requests.exceptions.ReadTimeout:
                print(f"⌛ 第 {page_index} 页请求超时，第 {attempt+1} 次重试")
                time.sleep(random.uniform(2, 5))
            except requests.exceptions.SSLError as e:
                print(f"⚠️ SSL错误，第 {page_index} 页，第 {attempt+1} 次重试: {e}")
                time.sleep(random.uniform(3, 6))
            except Exception as e:
                print(f"[请求失败] 第 {page_index} 页: {e}")
                time.sleep(random.uniform(1, 3))
                break
        return []


    def crawl_all_comments(self, max_page: int = 50, use_thread: bool = None, batch_size: int = 100):
        """
        抓取评论（自动判断是否使用多线程）
        - use_thread=None → 当 max_page >= 10 时自动切换到多线程
        - use_thread=True/False → 强制指定
        - batch_size → 多线程模式下每批提交的页数
        """
        if use_thread is None:
            use_thread = max_page >= 10

        if use_thread:
            self._crawl_threaded(max_page, batch_size=batch_size)
        else:
            self._crawl_sequential(max_page)
        
    def _process_comments(self, result, comments_list, new_count):
        """提取单页评论，并附加 poi 信息"""
        for data in result:
            cid = str(data.get("commentId", ""))
            if not cid or cid in self.seen_comment_ids:
                continue
            self.seen_comment_ids.add(cid)
            comments_list.append({
                "poi_id": self.poi_id,
                "poi_name": self.name,
                "poi_type": self.poi_type,
                "poi_subtype": self.poi_subtype,
                "commentId": cid,
                "content": data.get("content", "").replace("|", " ").replace("\n", " ").replace("\r", " "),
                "score": data.get("score", ""),
                "touristTypeDisplay": data.get("touristTypeDisplay", ""),
                "timeDuration": data.get("timeDuration", ""),
                "publishTypeTag": data.get("publishTypeTag", "")
            })
            new_count += 1
        return new_count

    def _crawl_threaded(self, max_page: int, batch_size: int = 100):
        print(f"\n[多线程] 开始抓取 {self.name} (POI: {self.poi_id}) 评论...")
        new_count = 0
        max_page = min(max_page, 200)

        for start in range(1, max_page + 1, batch_size):
            end = min(start + batch_size - 1, max_page)
            print(f"📦 提交任务：第 {start} 页 ~ 第 {end} 页")
            comments_list = []
            tasks = []

            with ThreadPoolExecutor(max_workers=POOL_NUMBER) as pool:
                for page_index in range(start, end + 1):
                    tasks.append(pool.submit(self.get_single_page_comments, page_index))

                for future in as_completed(tasks):
                    try:
                        result = future.result()
                        if result:
                            new_count = self._process_comments(result, comments_list, new_count)
                    except Exception as e:
                        print(f"任务执行异常: {e}")

            if comments_list:
                df = pd.DataFrame(comments_list)
                df.to_csv(self.file_path, mode="a", index=False, encoding="utf-8-sig",
                          header=not os.path.exists(self.file_path))
                print(f"✅ 批次完成：{self.name}，新增 {len(comments_list)} 条")
                time.sleep(1.5)

        print(f"🎉 {self.name} 评论采集完成：共新增 {new_count} 条，总数 {len(self.seen_comment_ids)} 条")
                
    def _crawl_sequential(self, max_page: int):
        print(f"\n [顺序] 开始抓取 {self.name} (POI: {self.poi_id}) 评论...")
        max_page = min(max_page, 11)
        comments_list = []
        new_count = 0
        empty_count = 0

        for page_index in range(1, max_page + 1):
            result = self.get_single_page_comments(page_index)
            if not result:
                empty_count += 1
                print(f"⚠️ 第 {page_index} 页为空（连续空页 {empty_count}）")
                if empty_count >= 3:
                    print("⛔ 连续 3 页无评论，提前终止抓取")
                    break
                continue
            else:
                empty_count = 0

            new_count = self._process_comments(result, comments_list, new_count)
            print(f"✅ 第 {page_index} 页抓取完成，累计新增 {new_count} 条评论")

        if comments_list:
            df = pd.DataFrame(comments_list)
            df.to_csv(self.file_path, mode="a", index=False, encoding="utf-8-sig",
                    header=not os.path.exists(self.file_path))
            time.sleep(1.5)
            print(f"📦 评论采集结束：共新增 {new_count} 条，总数 {len(self.seen_comment_ids)} 条")
        else:
            print(f"📭 {self.name} 未抓取到任何评论，未生成 CSV 文件")

#### 1.4 城市景点调度器

In [13]:
# 你的 cookie 列表
ctickets = [
    "A7F9A3EFD9AA82FFB0032491AE748FEDE7F3C0CDE1D26A99545E319742E6D2BD",
    "E89D6D1A5B866B002944528EAD21DD72A941215F8FEFEA4DC289217D0F196468"
]

# 城市名称 & 结果文件夹
city_name = "辽宁"
city_folder = os.path.join("comments_result", city_name)
os.makedirs(city_folder, exist_ok=True)

# 读取景点列表 CSV（含 poiId、name、comment_count）
df = pd.read_csv("携程景点_100061_辽宁_多关键词_去重.csv")

for _, row in df.iterrows():
    poi_id = int(row["景点ID"])
    poi_name = row["名称"]
    poi_type = row["大类"]
    poi_subtype = row["子类关键词"]
    comment_count = int(row["评论数"])

    
    # 估算页数：每页 20 条，向上取整
    estimated_page_count = math.ceil(comment_count / 20) + 1


    # 初始化爬虫实例
    spider = SpiderCtripComments(
        poi_id=poi_id,
        name=poi_name,
        poi_type=poi_type,
        poi_subtype=poi_subtype,
        folder=city_folder,
        ctickets=ctickets,
        sleep_time=0.3
    )


    # 自动判断是否用多线程（页数 ≥10 则多线程）
    spider.crawl_all_comments(max_page=estimated_page_count)



[多线程] 开始抓取 张学良旧居 (POI: 77702) 评论...
📦 提交任务：第 1 页 ~ 第 100 页
✅ 批次完成：张学良旧居，新增 2000 条
📦 提交任务：第 101 页 ~ 第 200 页
✅ 批次完成：张学良旧居，新增 1020 条
🎉 张学良旧居 评论采集完成：共新增 3020 条，总数 3020 条

[多线程] 开始抓取 沈阳故宫 (POI: 77696) 评论...
📦 提交任务：第 1 页 ~ 第 100 页
✅ 批次完成：沈阳故宫，新增 2000 条
📦 提交任务：第 101 页 ~ 第 200 页
✅ 批次完成：沈阳故宫，新增 1019 条
🎉 沈阳故宫 评论采集完成：共新增 3019 条，总数 3019 条

 [顺序] 开始抓取 爱新觉罗皇家博物院 (POI: 38963593) 评论...
✅ 第 1 页抓取完成，累计新增 12 条评论
⚠️ 第 2 页为空（连续空页 1）
📦 评论采集结束：共新增 12 条，总数 12 条

 [顺序] 开始抓取 沈阳故宫博物院-满清一条街 (POI: 39194924) 评论...
✅ 第 1 页抓取完成，累计新增 6 条评论
⚠️ 第 2 页为空（连续空页 1）
📦 评论采集结束：共新增 6 条，总数 6 条

 [顺序] 开始抓取 岁月流痕沈阳故宫博物院院史展 (POI: 52278778) 评论...
✅ 第 1 页抓取完成，累计新增 19 条评论
⚠️ 第 2 页为空（连续空页 1）
📦 评论采集结束：共新增 19 条，总数 19 条

 [顺序] 开始抓取 沈阳故宫博物院-十王亭 (POI: 18687057) 评论...
✅ 第 1 页抓取完成，累计新增 8 条评论
⚠️ 第 2 页为空（连续空页 1）
⚠️ 第 3 页为空（连续空页 2）
📦 评论采集结束：共新增 8 条，总数 8 条

[多线程] 开始抓取 新乐遗址博物馆 (POI: 77734) 评论...
📦 提交任务：第 1 页 ~ 第 18 页
✅ 批次完成：新乐遗址博物馆，新增 201 条
🎉 新乐遗址博物馆 评论采集完成：共新增 201 条，总数 201 条

[多线程] 开始抓取 大连贝壳博物馆 (POI: 10532785) 评论...
📦 提交任务：第 1 页 ~ 第 100 页
✅ 批次完成：大

#### 1.5 城市景点评论输出

In [ ]:
city_name = "重庆"
city_folder = os.path.join("comments_result", city_name)
# 存放合并后的文件路径
merged_file_path = os.path.join("comments_result", f"{city_name}_总评论表.csv")

# 获取该城市目录下所有 CSV 文件路径
csv_files = [f for f in os.listdir(city_folder) if f.endswith(".csv")]

# 存储所有 DataFrame 的列表
df_list = []

for file in csv_files:
    file_path = os.path.join(city_folder, file)
    try:
        df = pd.read_csv(file_path)
        # 添加额外列：景点名（从文件名中提取）
        df["景点名"] = file.replace(".csv", "")
        df_list.append(df)
    except Exception as e:
        print(f"⚠️ 读取 {file} 出错: {e}")

# 合并所有 DataFrame
if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.to_csv(merged_file_path, index=False, encoding="utf-8-sig")
    print(f"✅ 成功生成总表：{merged_file_path}，共 {len(merged_df)} 条评论")
else:
    print(f"❌ 未找到可合并的评论文件")

✅ 成功生成总表：comments_result\天津_总评论表.csv，共 21211 条评论


In [ ]:
# 查看是否还有重复的数据
import os
import pandas as pd

city_name = "上海"
city_folder = os.path.join("comments_result_补充", city_name)
merged_file_path = os.path.join("comments_result_补充", f"{city_name}_总评论表.csv")

# 获取城市目录下所有 CSV 文件
csv_files = [f for f in os.listdir(city_folder) if f.endswith(".csv")]

df_list = []

if csv_files:
    print(f"🗂️ 共找到 {len(csv_files)} 个文件：")
    for file in csv_files:
        print("   -", file)
        file_path = os.path.join(city_folder, file)
        try:
            df = pd.read_csv(file_path)
            df_list.append(df)
        except Exception as e:
            print(f"⚠️ 读取 {file} 出错: {e}")
else:
    print(f"❌ 未找到可合并的评论文件：{city_folder}")

# 合并并去重
if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)

    if "commentId" in merged_df.columns:
        before = len(merged_df)
        merged_df.drop_duplicates(subset="commentId", inplace=True)
        after = len(merged_df)
        print(f"🧹 已去除重复 commentId：共删除 {before - after} 条重复评论")

    merged_df.to_csv(merged_file_path, index=False, encoding="utf-8-sig")
    print(f"✅ 成功生成总表：{merged_file_path}，共 {len(merged_df)} 条评论")
else:
    print(f"❌ 无法合并，未读取到有效数据。")


🗂️ 共找到 371 个文件：
   - 351米太空舱.csv
   - JOLI ZOO(龙湖虹桥天街购物中心A馆店).csv
   - MOI幻觉艺术博物馆(虹桥丽宝广场店).csv
   - MrZoo-小小动物元室内动物园(上海嘉定万达中心).csv
   - Paradox Museum 悖论博物馆.csv
   - SNACKMUSEUM零食博物馆.csv
   - ZoolungZoolung萌宠游乐园(上海第一八佰伴店).csv
   - 《日月复光华》复旦大学 120 周年校庆展暨复旦大学艺术馆开馆展.csv
   - 上房园艺·梦花源.csv
   - 上济民间筷箸博物馆.csv
   - 上海东方地质科普馆.csv
   - 上海中医药博物馆.csv
   - 上海中华印刷博物馆.csv
   - 上海中国留学生博物馆.csv
   - 上海中国科举博物馆.csv
   - 上海交响音乐博物馆.csv
   - 上海交通大学(徐汇校区校史博物馆).csv
   - 上海交通大学(闵行校区)-图书馆(闵行校区).csv
   - 上海交通大学(闵行校区)-程及美术馆(闵行校区).csv
   - 上海交通大学-园林植物标本园(闵行校区).csv
   - 上海交通大学-植物园.csv
   - 上海人文历史地貌区.csv
   - 上海体育博物馆.csv
   - 上海体育博物馆服务中心.csv
   - 上海佘山国家森林公园.csv
   - 上海儿童动物园.csv
   - 上海儿童博物馆.csv
   - 上海公安博物馆.csv
   - 上海共青森林公园.csv
   - 上海农垦博物馆.csv
   - 上海凝聚力工程博物馆.csv
   - 上海制冷历史博物馆.csv
   - 上海动漫博物馆.csv
   - 上海动物园-两栖爬虫动物馆.csv
   - 上海动物园.csv
   - 上海动物园_all_unique.csv
   - 上海动画博物馆.csv
   - 上海博物馆(东馆).csv
   - 上海博物馆.csv
   - 上海博物馆_all_unique.csv
   - 上海印刷博物馆.csv
   - 上海历史博物馆-古代上海.csv
   - 上海历史收藏馆.csv
   - 上海国际酒文化博物馆.csv
  